#  AccessGuru Detect Notebook
Full Pipline:
1. **AccessGuruDetect SyntaxLayout**: Detect violations (Axe-Playwright) [Notebook Link](https://colab.research.google.com/drive/1edKtrSCJ2FrZqU0G8v9424yKG-qQrrRU?usp=sharing)
2. **AccessGuruDetect Semantic**: Detect violations (LLM)[Current Notebook]
3. **AccessGuruCorrect**: Generate corrections using LLM prompting strategies. [Notebook Link](https://colab.research.google.com/drive/1zoW8fL6VLz1sE8BoHbfnIaaOrgMeNKC5?usp=drive_link)

This notebook demonstrates a full pipeline for **AccessGuruDetect**: Detect violations (Axe-Playwright + LLM)
We’ll walk through each step with explanations and runnable code.

# 1. AccessGuruDetect
We implemented the AccessGuruDetect using
Axe-Playwright-1.51.0 for syntax and layout accessibility
violations.

## 1.1. Install Dependencies
Use "pip install" to install the package

In [ ]:
!pip install playwright
!playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 11.1 MB/s eta 0:00:00
173.7 MiB [] 0% 0.0s173.7 MiB [] 0% 52.9s173.7 MiB [] 0% 23.8s173.7 MiB [] 0% 15.0s173.7 MiB [] 0% 7.7s173.7 MiB [] 1% 5.0s173.7 MiB [] 2% 3.9s173.7 MiB [] 3% 3.2s173.7 MiB [] 4% 2.8s173.7 MiB [] 5% 2.9s173.7 MiB [] 5% 2.7s173.7 MiB [] 6% 2.6s173.7 MiB [] 7% 2.6s173.7 MiB [] 8% 2.4s173.7 MiB [] 9% 2.4s173.7 MiB [] 9% 2.3s173.7 MiB [] 10% 2.2s173.7 MiB [] 12% 2.1s173.7 MiB [] 13% 2.0s173.7 MiB [] 14% 1.9s173.7 MiB [] 16% 1.8s173.7 MiB [] 17% 1.7s173.7 MiB [] 18% 1.7s173.7 MiB [] 19% 1.6s173.7 MiB [] 20% 1.6s173.7 MiB [] 22% 1.5s173.7 MiB [] 23% 1.5s173.7 MiB [] 24% 1.4s173.7 MiB [] 25% 1.4s173.7 MiB [] 25% 1.5s173.7 MiB [] 26% 1.4s173.7 MiB [] 28% 1.4s173.7 MiB [] 29% 1.3s173.7 MiB [] 31% 1.3s173.7 MiB [] 32% 1.2s173.7 MiB [] 34% 1.2s173.7 MiB [] 35% 1.1s173.7 MiB [] 37% 1.1s173.7 MiB [] 38% 1.1s173.7 MiB [] 39% 1.0s173.7 MiB [] 40% 1.0s173.7 MiB [] 42% 1.0s173.7 MiB [] 43% 0.9s173.7 MiB [] 44% 0.9s173.7 MiB 

In [ ]:
! pip install wget
! pip install selenium

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=71e47171c926e7210ba631335c2a11487793961d99f0535c011823934a2ec00c
  Stored in directory: /root/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.15.0
    Uninstalling typing_extensions-4.15.0:
      Successfully uninstalled typing_extensions-4.15.0


## 1.2. Imports & Setup

In [ ]:
import os
import re
import json

import wget
import requests
import aiohttp
import asyncio
import nest_asyncio
import pandas as pd
import base64
from pathlib import Path
from datetime import datetime
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from playwright.async_api import async_playwright

nest_asyncio.apply()


In [ ]:
# Output directories
output_dir = "/content/html_pages_async"
screenshot_dir = "/content/element_screenshots"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(screenshot_dir, exist_ok=True)

In [ ]:
# Download required data(violation taxonomy, mapping dictionary) from AccessGuru Repo
! wget 'https://raw.githubusercontent.com/NadeenAhmad/AccessGuruLLM/refs/heads/main/data/prompts_support/violation_taxonomy.csv'
! wget 'https://raw.githubusercontent.com/NadeenAhmad/AccessGuruLLM/refs/heads/main/data/prompts_support/mapping_dict_file.json'
! wget 'https://raw.githubusercontent.com/NadeenAhmad/AccessGuruLLM/refs/heads/main/data/prompts_support/violations_short_description.json'

In [ ]:
mapping_dict_path = '/content/mapping_dict_file.json'
with open(mapping_dict_path, 'r') as file:
  mapping_dict = json.load(file)

violation_description_path = '/content/violations_short_description.json'
with open(violation_description_path, 'r') as file:
  violation_description_dict = json.load(file)

taxonomy_path = "/content/violation_taxonomy.csv"
cat_data = pd.read_csv(taxonomy_path)


In [ ]:
impactScore = {
  "critical": 5,
  "serious": 4,
  "moderate": 3,
  "minor": 2,
  "cosmetic": 1,
}

impact_dict = {
      'image-alt-not-descriptive': 'critical',
      'video-captions-not-descriptive': 'critical',
      'lang-mismatch': 'serious',
      'missing-lang-tag': 'serious',
      'link-text-mismatch': 'serious',
      'button-label-mismatch': 'critical',
      'form-label-mismatch': 'critical',
      'ambiguous-heading': 'moderate',
      'incorrect-semantic-tag': 'serious',
      'landmark-structural-violation': 'serious',
      'landmark-purpose-mismatch': 'serious',
      'page-title-not-descriptive': 'serious',
      'autocomplete-purpose-mismatch': 'serious',
      'color-only-distinction': 'serious',
      'illogical-focus-order': 'serious',
      'label-name-mismatch': 'serious'
       }

## 1.3. Utility Functions
modules needed for the Detection:
*   Download images,
*   Check if given URL can be scraped
*   save scraped HTML code,
*   supplementary information extraction .

In [ ]:
async def save_html(html, url):
    parsed = urlparse(url)
    netloc = parsed.netloc.replace(".", "_")
    path = parsed.path.strip("/") or "home"
    path = "".join([c if c.isalnum() else "_" for c in path])
    file_name = f"{netloc}_{path}.html"
    file_path = os.path.join(output_dir, file_name)

    with open(file_path, "w", encoding="utf-8") as f:
        f.write(html)

    return file_path

async def url_check(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()

        try:
            # Try navigating with a timeout
            response = await page.goto(url, timeout=15000, wait_until="domcontentloaded")

            if not response:
                print(f'No response for {url}. Please try another URL')
                return "not scraped"

            status = response.status
            final_url = page.url

            if status >= 400:
                print(f"Failed to load {url} (status {status}). Please try another URL")
                return "not scraped"

            print(f"Loaded {final_url} (status {status})")
            scrape_status = "scraped"
        except Exception as e:
            print(f"Error scraping {url}. Please try another URL")
            return "not scraped"
        finally:
            await browser.close()


def find_matching_ul(soup, snippet_html):
    snippet_soup = BeautifulSoup(snippet_html, 'html.parser')
    snippet_ul = snippet_soup.find('ul')
    if not snippet_ul:
        return None

    snippet_classes = set(snippet_ul.get('class', []))

    for ul in soup.find_all('ul'):
        ul_classes = set(ul.get('class', []))
        if snippet_classes.issubset(ul_classes):
            return str(ul)

    return None


def get_landmark_container_for_tag(soup, tag_name='main'):
    tag = soup.find(lambda tag: tag.name == tag_name or tag.get('role', '').lower() == tag_name)
    if not tag:
        return None, f"No <{tag_name}> tag or role='{tag_name}' found"

    landmark_roles = {'banner', 'complementary', 'main', 'contentinfo', 'navigation', 'region'}
    current = tag.parent

    while current:
        role = current.get('role', '').lower()
        if role in landmark_roles or current.name in landmark_roles:
            return current, None
        current = current.parent if hasattr(current, 'parent') else None

    return tag, None


def role_or_tag(role_value, tag_name):
    return lambda tag: tag.name == tag_name or tag.attrs.get("role") == role_value

def get_full_list_html(web_html: str, affected_html: str) -> str | None:
    soup = BeautifulSoup(web_html, "html.parser")

    # Parse the affected HTML to extract the tag and attributes
    affected_soup = BeautifulSoup(affected_html, "html.parser")
    affected_element = affected_soup.find()

    if not affected_element:
        print("Could not parse affected HTML")
        return None

    # Find matching element in full page HTML
    matches = soup.find_all(affected_element.name, attrs=affected_element.attrs)

    for match in matches:
        # Return the outer HTML of the matching list
        if match.name in ['ul', 'ol']:
            return str(match)

    print("No matching full list element found.")
    return None


# --- Parse <style> blocks into a {selector -> {prop: value}} index ---
def parse_css_rules_from_style_tags(full_html: str):
    soup = BeautifulSoup(full_html, "html.parser")
    css_text = "\n".join(st.get_text() for st in soup.find_all("style"))
    rules = {}  # selector -> {prop: value}

    # Very simple CSS parser: selector { prop:value; ... }
    for selectors, props in re.findall(r'([^{]+)\{([^}]+)\}', css_text, re.DOTALL):
        # parse props
        props_dict = {}
        for k, v in re.findall(r'([-\w]+)\s*:\s*([^;]+);?', props):
            props_dict[k.strip().lower()] = v.strip()
        # split combined selectors: #id, .class, span, etc.
        for sel in selectors.split(","):
            sel = sel.strip()
            if sel:
                # last rule wins; overwrite
                rules[sel] = {**rules.get(sel, {}), **props_dict}
    return rules

# --- Inline style -> dict ---
def parse_inline_style(style_str: str):
    out = {}
    for k, v in re.findall(r'([-\w]+)\s*:\s*([^;]+);?', style_str or ""):
        out[k.strip().lower()] = v.strip()
    return out

# --- Try to locate the snippet tag inside the full DOM (id > class > text) ---
def locate_in_full_html(snippet_tag, full_soup: BeautifulSoup):
    if snippet_tag.has_attr("id"):
        found = full_soup.find(id=snippet_tag["id"])
        if found: return found

    if snippet_tag.has_attr("class"):
        # exact class set match first
        found = full_soup.find(snippet_tag.name, class_=snippet_tag.get("class"))
        if found: return found
        # fallback: any element with any of those classes
        for cls in snippet_tag.get("class"):
            found = full_soup.find(snippet_tag.name, class_=lambda c: c and cls in c)
            if found: return found

    text_content = snippet_tag.get_text(strip=True)
    if text_content:
        # match by tag + text content
        found = full_soup.find(snippet_tag.name, string=re.compile(re.escape(text_content)))
        if found: return found

    return None  # not found

# --- Resolve color from inline + stylesheet rules for a single Tag ---
def resolve_color_for_tag(tag, css_rules: dict):
    # Inline first (highest priority)
    inline = parse_inline_style(tag.get("style", "")) if tag.has_attr("style") else {}
    inline_color = inline.get("color")
    inline_bg   = inline.get("background-color")

    # Candidates (rough cascade): #id > .class > tag
    candidates = []
    if tag.has_attr("id"):
        candidates.append(f"#{tag['id']}")
    if tag.has_attr("class"):
        for cls in tag["class"]:
            candidates.append(f".{cls}")
            candidates.append(f"{tag.name}.{cls}")  # sometimes defined as tag.class
    candidates.append(tag.name)

    css_color = None
    css_bg    = None
    for sel in candidates:
        if sel in css_rules:
            # last matching selector wins (iterate in order above)
            css_color = css_rules[sel].get("color", css_color)
            css_bg    = css_rules[sel].get("background-color", css_bg)

    # Tailwind-like utility classes (optional tokens only)
    tailwind_tokens = []
    if tag.has_attr("class"):
        tailwind_tokens = [c for c in tag["class"] if c.startswith(("text-", "bg-", "dark:text-", "dark:bg-"))]

    return {
        "inline_color": inline_color,
        "inline_background_color": inline_bg,
        "css_color": css_color,
        "css_background_color": css_bg,
        "class_color_tokens": tailwind_tokens,
    }

# --- Pull everything together for your violation branch ---
def extract_colors_for_affected_elements(affected_html_str: str, full_html: str):
    # parse the snippet string into Tag objects
    s_soup = BeautifulSoup(affected_html_str, "html.parser")
    snippet_tags = s_soup.find_all()

    # parse the full page once
    full_soup = BeautifulSoup(full_html, "html.parser")
    css_rules = parse_css_rules_from_style_tags(full_html)

    results = []
    for snip in snippet_tags:
        # find the real element in the full DOM (so we get actual id/class/style)
        real = locate_in_full_html(snip, full_soup) or snip  # fallback to snippet itself
        colors = resolve_color_for_tag(real, css_rules)

        results.append({
            "snippet": str(snip),
            "resolved_element": str(real),
            **colors
        })
    return results


def parse_css_variables(soup):
    """Extract all CSS variables from <style> blocks."""
    # soup = BeautifulSoup(full_html, "html.parser")
    css_text = "\n".join(st.get_text() for st in soup.find_all("style"))
    variables = {}

    # Match --variable-name: value;
    for var, val in re.findall(r'--([-\w]+)\s*:\s*([^;]+);', css_text):
        variables[f'--{var}'] = val.strip()
    return variables

def resolve_color_value(color_value, css_variables):
    """Replace var(--variable) with actual value if present."""
    if not color_value:
        return None
    # simple var() replacement
    matches = re.findall(r'var\((--[\w-]+)\)', color_value)
    for var in matches:
        if var in css_variables:
            color_value = color_value.replace(f'var({var})', css_variables[var])
    return color_value


def download_images_from_snippets(snippets, save_dir="supplementary_images"):
    os.makedirs(save_dir, exist_ok=True)
    paths = []

    for idx, snippet in enumerate(snippets):
        # Parse snippet to extract src
        soup = BeautifulSoup(str(snippet), "html.parser")
        img = soup.find("img")
        if not img or not img.get("src"):
            continue

        url = img["src"]

        # Get file extension (default to .jpg)
        ext = os.path.splitext(url.split("?")[0])[1]
        if not ext:
            ext = ".jpg"
        # Build save path
        filename = f"image_{idx}{ext}"
        filepath = os.path.join(save_dir, filename)

        try:
            response = requests.get(url, timeout=15)
            response.raise_for_status()
            with open(filepath, "wb") as f:
                f.write(response.content)
            paths.append(filepath)
        except Exception as e:
            print(f"Error downloading {url}: {e}")

    return paths


async def extract_supplementary_info(row):
    # Skip if supplementary_information already exists and is non-empty
    if pd.notna(row.get("supplementary_information")) and str(row.get("supplementary_information")).strip():
        return row["supplementary_information"]

    violation = row["violation_name"]
    html_file = row["html_file_name"]

    if "content" not in html_file_name:
        html_file = "/content/"+str(html_file)

    if not html_file.endswith(('.html', '.txt')):
        html_file += '.html'

    snippet = row["affected_html_elements"]

    try:
        with open(html_file, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f, 'lxml')
    except Exception as e:
        return f"HTML load error: {e}"

    # ---------- Violation-Specific Logic ----------
    # Supplementary Information: Color Violations
    if violation in ['color-only-distinction', 'color-contrast-enhanced', 'color-contrast']:
        # --- Parse snippet HTML into tags ---
        snippet_tags = row["affected_html_elements"]
        if type(snippet_tags) != list:
            s_soup = BeautifulSoup(row["affected_html_elements"], "html.parser")
            snippet_tags = s_soup.find_all()

        # --- Parse full HTML and CSS variables ---
        # full_soup = BeautifulSoup(html, "html.parser")
        with open(str(html_file), "r", encoding="utf-8") as f:
            html = f.read()

        # Parse with BeautifulSoup
        full_soup = BeautifulSoup(html, "html.parser")
        css_variables = parse_css_variables(full_soup)

        inline_color = set()
        inline_background_color = set()
        class_color_tokens = set()
        # --- Iterate over snippets ---
        for snip in snippet_tags:

            # Try to find the same element in the full HTML (by id > class > text)
            real = None
            if snip.has_attr("id"):
                real = full_soup.find(id=snip["id"])
            if not real and snip.has_attr("class"):
                real = full_soup.find(snip.name, class_=snip.get("class"))
            if not real:
                text_content = snip.get_text(strip=True)
                if text_content:
                    real = full_soup.find(snip.name, string=re.compile(re.escape(text_content)))
            if not real:
                real = snip  # fallback to snippet itself

            # --- Extract inline color ---
            inline_style = real.get("style", "")
            inline_colors = {}
            for prop, val in re.findall(r'([-\w]+)\s*:\s*([^;]+);?', inline_style):
                prop = prop.lower()
                inline_colors[prop] = resolve_color_value(val.strip(), css_variables)

            # --- Extract class-based color tokens ---
            class_colors = []
            if real.has_attr("class"):
                class_colors = [c for c in real["class"] if c.startswith(("text-", "bg-", "dark:text-", "dark:bg-"))]

            if inline_colors.get("color") or inline_colors.get("background-color") or class_colors:
                # "affected_element": str(snip),
                # "resolved_element": str(real),
                inline_color.add(inline_colors.get("color"))
                inline_background_color.add(inline_colors.get("background-color"))
                for i in class_colors:
                    class_color_tokens.add(i)
        return {
            "inline_color":(inline_color),
            "inline_background_color":(inline_background_color),
            "class_color_tokens":(class_color_tokens)
        }
    # Supplementary Information: Image violations -> get image source from affected html and take a screenshot
    if violation in [
        "image-alt", "input-image-alt", "image-alt-not-descriptive",
        "image-redundant-alt", "area-alt", "frame-title", "frame-title-unique",
        "object-alt", "role-img-alt", "svg-img-alt", "button-name", "input-button-name"
    ]:
        # s_soup = BeautifulSoup(row["affected_html_elements"], "html.parser")
        # snippet_tags = s_soup.find_all()
        downloaded_paths = download_images_from_snippets(row["affected_html_elements"])
        return downloaded_paths


    # Supplementary Information: link-name
    if violation in ["link-name","link-text-mismatch"]:
        link_info_list = []

        snippets = row["affected_html_elements"]
        if type(snippets) != list:
            snippets = re.findall(r'<a [^>]+>', snippet)

        for snippet in snippets:
            affected_html = snippet.strip()

            href_match = re.search(r'href=["\']([^"\']+)["\']', affected_html)
            target_match = re.search(r'target=["\']([^"\']+)["\']', affected_html)

            href = href_match.group(1) if href_match else None
            explicit_target = target_match.group(1).lower() if target_match else None

            if not href or not href.startswith("http"):
                continue

            try:
                async with async_playwright() as p:
                    browser = await p.chromium.launch()
                    page = await browser.new_page()
                    await page.goto(href, timeout=15000)

                    # Get page title
                    page_title = await page.title()
                    if not page_title:
                        html = await page.content()
                        soup = BeautifulSoup(html, "html.parser")
                        page_title = soup.title.string.strip() if soup.title else "No title found"

                    link_info_list.append(
                        f"The title of the target {href} link page: {page_title}"
                    )

                    await browser.close()

            except Exception as e:
                print(f"Error processing link '{href}': {e}")

        return "\n\n".join(link_info_list) if link_info_list else ""

    # Supplementary Information: List
    if violation == "list":
        affected_html = snippet
        full_list_html = get_full_list_html(soup, snippet)
        if full_list_html:
            return full_list_html
        else:
            return ""


    elif any(v in violation for v in ["ambiguous-heading", "empty-heading", "heading-order"]):
        headings = soup.find_all(re.compile(r'^h[1-6]$'))
        results = []

        for heading in headings:
            if not heading.get_text(strip=True):
                next_elements = []
                sibling = heading.find_next_sibling()
                while sibling and len(next_elements) < 3:
                    if sibling.name in ["p", "ul", "ol", "div", "section"]:
                        next_elements.append(str(sibling))
                    sibling = sibling.find_next_sibling()
                results.append(f"{str(heading)}\n\n" + "\n\n".join(next_elements))

        return "\n\n---\n\n".join(results) if results else ""

    elif "empty-table-header" in violation:
        headers = soup.find_all("th")
        results = []

        for th in headers:
            if not th.get_text(strip=True):
                next_elements = []
                sibling = th.find_next_sibling()
                while sibling and len(next_elements) < 3:
                    if sibling.name in ["td", "th", "tr"]:
                        next_elements.append(str(sibling))
                    sibling = sibling.find_next_sibling()
                results.append(f"{str(th)}\n\n" + "\n\n".join(next_elements))

        return "\n\n---\n\n".join(results) if results else ""

    elif "page-has-heading-one" in violation:
        title_html = str(soup.title) if soup.title and soup.title.string else ""
        h1_tags = soup.find_all("h1")
        h1_html = "\n\n".join(str(h) for h in h1_tags[:3]) if h1_tags else ""
        return f"{title_html}\n\n---\n\n{h1_html}"

    elif "page-title-not-descriptive" in violation:
        title_html = str(soup.title) if soup.title and soup.title.string else ""
        headings = soup.find_all(re.compile(r"^h[1-6]$"))
        heading_html = [str(h) for h in headings[:10]]
        return f"{title_html}\n\n---\n\n" + "\n\n".join(heading_html) if heading_html else title_html

    elif "document-title" in violation:
        title_html = str(soup.title) if soup.title and soup.title.string and soup.title.string.strip() else ""
        # title_html = str(soup.title) if soup.title and soup.title.string.strip() else ""
        headings = soup.find_all(re.compile(r"^h[1-6]$"))
        heading_html = [str(h) for h in headings[:10]]
        return f"{title_html}\n\n---\n\n" + "\n\n".join(heading_html) if heading_html else title_html

    elif any(v in violation for v in [
        "duplicate-id", "duplicate-id-aria", "duplicate-id-active",
        "landmark-no-duplicate-contentinfo", "landmark-no-duplicate-main",
        "landmark-no-duplicate-banner", "landmark-unique"
    ]):
        report = []

        # Duplicate ID check
        if any(v in violation for v in ["duplicate-id", "duplicate-id-aria", "duplicate-id-active"]):
            id_map = {}
            for tag in soup.find_all(attrs={"id": True}):
                id_map.setdefault(tag["id"], []).append(tag)

            duplicates = {k: v for k, v in id_map.items() if len(v) > 1}
            for dup_id, elements in list(duplicates.items())[:5]:
                report.append(f"ID '{dup_id}' is used {len(elements)} times:")
                for el in elements[:3]:
                    snippet = str(el)
                    report.append(snippet if len(snippet) <= 500 else snippet[:500] + "...")

        # Duplicate landmarks
        if "landmark-no-duplicate-contentinfo" in violation:
            contentinfos = soup.find_all(role_or_tag("contentinfo", "footer"))
            if len(contentinfos) > 1:
                report.append(f"{len(contentinfos)} <footer> or role='contentinfo' elements found:\n" +
                              "\n---\n".join(str(tag) for tag in contentinfos))

        if "landmark-no-duplicate-main" in violation:
            mains = soup.find_all(role_or_tag("main", "main"))
            if len(mains) > 1:
                report.append(f"{len(mains)} <main> or role='main' elements found:\n" +
                              "\n---\n".join(str(tag) for tag in mains))

        if "landmark-no-duplicate-banner" in violation:
            banners = soup.find_all(role_or_tag("banner", "header"))
            if len(banners) > 1:
                report.append(f"{len(banners)} <header> or role='banner' elements found:\n" +
                              "\n---\n".join(str(tag) for tag in banners))

        if "landmark-unique" in violation:
            roles = ["main", "banner", "contentinfo", "navigation", "search", "complementary", "form"]
            for role in roles:
                tags = soup.find_all(attrs={"role": role})
                if len(tags) > 1:
                    report.append(f"Role '{role}' found {len(tags)} times:\n" +
                                  "\n---\n".join(str(tag) for tag in tags))

        return "\n\n".join(report) if report else ""

    elif violation in [
        "landmark-main-is-top-level", "landmark-banner-is-top-level", "landmark-complementary-is-top-level"
    ]:
        tag_map = {
            "landmark-main-is-top-level": "main",
            "landmark-banner-is-top-level": "banner",
            "landmark-complementary-is-top-level": "complementary"
        }
        tag_role = tag_map.get(violation, "main")
        container, error = get_landmark_container_for_tag(soup, tag_role)
        return str(container) if container else ""

    elif any(v in violation for v in [
        "lang-mismatch", "missing-lang-tag", "html-lang-valid",
        "html-xml-lang-mismatch", "valid-lang", "html-has-lang"
    ]):
        title = soup.title.string.strip() if soup.title and soup.title.string else "No <title> tag or title is empty"
        headings = soup.find_all(re.compile(r'^h[1-6]$'))
        heading_texts = [f"{h.name.upper()}: {h.get_text(strip=True)}" for h in headings if h.get_text(strip=True)]
        return f"Title: {title} | Headings: {' | '.join(heading_texts[:10])}" if heading_texts else f"Title: {title}"

    return ""



## 1.5. AccessGuru Detect

## 1.5.b AcessGuru Detect for semantic accessibility violations

For Semantic Detec, we will use the free model Qwen: https://openrouter.ai/qwen/qwen2.5-vl-72b-instruct:free to test. <br>
**PLEASE NOTE** THAT THIS MODEL DOESN'T WORK WELL. PLEASE USE GPT-4o AS MENTIONED IN THE PAPER FOR AccessGuru Semantic Detection.


### API Access Guide
1. Open https://openrouter.ai/settings/keys
2. Click on "Create API key"
4. Create the api and copy the key to the clipboard

In [ ]:
API_KEY = "sk-or-v1-xxxxxx"  # Replace with your actual API key

Openrouter_API_URL = "https://openrouter.ai/api/v1/chat/completions"

In [ ]:
async def url_check_AndHtml(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()

        try:
            # Try navigating with a timeout
            response = await page.goto(url, timeout=15000, wait_until="domcontentloaded")

            if not response:
                print(f'No response for {url}. Please try another URL')
                return "not scraped",None,None

            status = response.status
            final_url = page.url

            if status >= 400:
                print(f"Failed to load {url} (status {status}). Please try another URL")
                return "not scraped",None,None

            print(f"Loaded {final_url} (status {status})")
            html = await page.content()
            html_file_name = await save_html(html, url)  # Save the HTML
            return "scraped",html,html_file_name
        except Exception as e:
            print(f"Error scraping {url}. Please try another URL")
            return "not scraped",None,None
        finally:
            await browser.close()

def encode_image_to_data_url(image_path: Path) -> str:
    with open(image_path, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode("utf-8")
        image_url = f"data:image/png;base64,{encoded_image}"
    return image_url

def generate_semantic_prompt(domain,url,taxonomy,html_text):
    SEMANTIC_DETECT_PROMPT_TEMPLATE = """ You are a web accessibility expert. Your task is to detect semantic accessibility violations in the given HTML Web page. These violations are often not detectable by standard automated tools and require interpretation of the content meaning and user context.

      A semantic violation occurs when:
      - Attributes like alt text, language, or link/button labels are present but do not provide meaningful or accurate information,
      - Visual or multimedia content is not described in a way that conveys its purpose to users with disabilities.

      Use the information below to guide your analysis, you are operating on:
      - The domain of the web page: {Insert Web page Domain}
      - The URL of the web page: {Insert Web page URL}

      You are provided with:
      - The HTML code of the web page to analyze,
      - The full semantic violation taxonomy. This taxonomy defines specific types of semantic violations, their descriptions,
      - A screenshot of the rendered view of the web page.

      {Semantic Violation Taxonomy}

      {Insert HTML here}
      {Insert Web page screenshot}


      Now, review the HTML and supplementary data. List all semantic violations you detect, and for each:
      1. Identify the affected HTML element. Enclose the exact
      HTML snippet using the markers [START] and [END].
      2. Specify the violation name.
      """

    full_prompt = SEMANTIC_DETECT_PROMPT_TEMPLATE.replace(
        "{Insert Web page Domain}", domain
    ).replace(
        "{Insert Web page URL}", url
    ).replace(
        "{Semantic Violation Taxonomy}", taxonomy
    ).replace(
        "{Insert HTML here}", html_text
    ).replace(
        "{Insert Web page screenshot}", "(Screenshot attached below)."
    )
    return full_prompt

def generate_response(prompt_text,screenshot_data_url):
    message = [{'role': 'system', 'content': "You are a web accessibility expert."},
              {'role': 'user', 'content': [
                          {"type": "text", "text": prompt_text},
                          {
                              "type": "image_url",
                              "image_url": {
                                  "url": screenshot_data_url
                              },
                          },
                        ],
                    }
              ]

    response = requests.post(
        Openrouter_API_URL,
        headers={
            "Authorization": f"Bearer {API_KEY}",
            "Content-Type": "application/json"
        },
        json={
            "model": "qwen/qwen2.5-vl-72b-instruct:free",
            "messages": message
        }
    )

    if response.status_code == 200:
        chat_response = response.json()["choices"][0]["message"]["content"]
        return chat_response
    else:
        return f"Error: {response.status_code}, {response.text}"

def post_process_response(web_URL, url_df, text: str, full_html: str):
    """
    Extract all violations from LLM output into a list of dicts
    with 'element' and 'violation' keys.
    If the same violation_name occurs, append the element to its list.
    """
    violations = []
    violation_index = {}  # map violation_name -> index in violations list

    # Regex to find each violation block (element + violation name)
    pattern = re.compile(
        r"\[START\]\s*```html\s*(.*?)\s*```\s*\[END\].*?Violation Name:\**\s*([^\n]+)",
        re.DOTALL | re.IGNORECASE
    )

    matches = pattern.findall(text)
    count = 0
    web_URL_id = url_df[url_df["web_URL"] == web_URL]["web_URL_id"].iloc[0]

    for element, violation in matches:
        element_clean = element.strip()

        # Check if element exists in full HTML
        if element_clean not in full_html:
            continue  # skip if not found

        violation_name = violation.strip().replace("`", "")

        try:
            violation_description = violation_description_dict[violation_name]
        except KeyError:
            violation_description = ""

        violation_impact = impact_dict.get(violation_name, "Unknown")

        if violation_name in violation_index:
            # Append to existing entry
            idx = violation_index[violation_name]
            violations[idx]["affected_html_elements"].append(element_clean)
        else:
            # Create new entry
            new_id = f"{web_URL_id}_{count}"
            violations.append({
                "id": new_id,
                "web_URL": web_URL,
                "affected_html_elements": [element_clean],  # store as list
                "violation_name": violation_name,
                "violation_description": violation_description,
                "violation_description_url": "https://github.com/NadeenAhmad/AccessGuruLLM/blob/main/taxonomy_web_accessibility_violations.md",
                "violation_impact": violation_impact,
                "violation_score": impactScore[violation_impact]
            })
            violation_index[violation_name] = len(violations) - 1
            count += 1

    return violations, count


async def process_dataframe(df):
    results = []
    for _, row in df.iterrows():
        supplementary_info = await extract_supplementary_info(row)
        results.append(supplementary_info)
    return results



## 1.6. Example Run


## 1.6.b. Semantic Example Run
The input should have the following values for each keys:
*   **web_URL_id** : Unique identifier for the URL
*   **domain_category** : The domain of the website's subject area (Domains: Educational Platforms, Government and Public Services, News and Media, E-commerce, Streaming Platforms, Health and Wellness, Technology, Science and Research )
*   **web_URL** : The URL of the webpage where the violation was found
*   **screenshot_path**: Path to the captured screenshot of the Webpage. <br>
Please take the screenshot of the webPage with a fixed viewport width of 1440 pixels and a height equal to the full scrollable length of the page, resulting in an image with a height up to 1440 times the viewport height, depending on page length.

**Input dictionary example**:
```
{'web_URL_id':1, 'web_URL':'https://www.ki.uni-stuttgart.de/', 'domain_category': 'Educational Platforms','screenshot_path':"/content/screencapture-ki-uni-stuttgart-de-2025-08-22-03_25_56.png"}

```

In [ ]:
sem_df = pd.read_csv("/content/violation_taxonomy.csv")
sem_df = sem_df[sem_df["Category"]==" Semantic"]
sem_violations = sem_df.violationnumberID.values
sem_violations_list = [line.strip() for line in sem_violations if line.strip()]
taxonomy = str(sem_violations_list)

# RealWorld web_URL
# After taking the screenshot of the webpage, PLEASE CHANGE THE screenshot_path.
input_dict = {'web_URL_id':1, 'web_URL':'https://www.ki.uni-stuttgart.de/', 'domain_category': 'Educational Platforms','screenshot_path':"/content/screencapture-ki-uni-stuttgart-de-2025-08-22-03_25_56.png"}

# Simple web_URL
# After taking the screenshot of the webpage, PLEASE CHANGE THE screenshot_path.
# input_dict = {'web_URL_id':1, 'web_URL':"https://www.w3.org/WAI/content-assets/wcag-act-rules/testcases/qt1vmo/485f10faf222cd48fea2ab3ee79c2d354e51ea33.html",'domain_category': 'Educational Platforms','screenshot_path':"/content/Screenshot 2025-08-22 at 5.11.59 AM.png"}

url_df = pd.DataFrame([input_dict], columns=list(input_dict.keys()))
urls = list(url_df["web_URL"].values)

output = pd.DataFrame()
for url in urls:
    scrape_status,html,html_file_name =  await url_check_AndHtml(url)

    if scrape_status == "not scraped":
       break
    else:
      url_df["scrape_status"] = scrape_status
      url_df["html_file_name"] = html_file_name

      screenshot_path = url_df[url_df["web_URL"]==url]["screenshot_path"][0]
      screenshot_data_url = encode_image_to_data_url(screenshot_path)

      domain_category = url_df[url_df["web_URL"]==url]["domain_category"][0]
      prompt_text = generate_semantic_prompt(domain_category,url,taxonomy,html)
      llm_response = generate_response(prompt_text,screenshot_data_url)

      violations,violation_count = post_process_response(url,url_df,llm_response,html)
      url_df["violation_count"] = violation_count
      for each_violation in violations:
          df_dictionary = pd.DataFrame([each_violation])
          output = pd.concat([output, df_dictionary], ignore_index=True)


      if len(output)>0:
          violation_df = pd.merge(output, url_df, on="web_URL")
          violation_df["wcag_reference"] = violation_df["violation_name"].map(mapping_dict)
          # violation_df["supplementary_information"]  = ""
          violation_df["violation_category"]  = "Semantic"
          violation_df = violation_df[violation_df['violation_count'] != 0]


violation_df.head()

Loaded https://www.ki.uni-stuttgart.de/ (status 200)


,id,web_URL,affected_html_elements,violation_name,violation_description,violation_description_url,violation_impact,violation_score,web_URL_id,domain_category,screenshot_path,scrape_status,html_file_name,violation_count,wcag_reference,violation_category
0,1_0,https://www.ki.uni-stuttgart.de/,"[<img src=""https://www.ki.uni-stuttgart.de/img...",image-alt-not-descriptive,Inaccurate or misleading alternative text that...,https://github.com/NadeenAhmad/AccessGuruLLM/b...,critical,5,1,Educational Platforms,/content/screencapture-ki-uni-stuttgart-de-202...,scraped,/content/html_pages_async/www_ki_uni-stuttgart...,2,[1.1.1 Non-text Content],Semantic
1,1_1,https://www.ki.uni-stuttgart.de/,"[<a href=""https://www.ki.uni-stuttgart.de/inst...",link-text-mismatch,Links fail to convey their purpose or are ambi...,https://github.com/NadeenAhmad/AccessGuruLLM/b...,serious,4,1,Educational Platforms,/content/screencapture-ki-uni-stuttgart-de-202...,scraped,/content/html_pages_async/www_ki_uni-stuttgart...,2,"[2.4.4 Link Purpose (In Context), 2.4.9 Link P...",Semantic


# Extract Supplementary Informations

In [ ]:
# violation_df = pd.read_csv("/content/test.csv") # to test the existing or saved files
results = asyncio.run(process_dataframe(violation_df))
violation_df["supplementary_information"] = results
violation_df.head()

,id,web_URL,affected_html_elements,violation_name,violation_description,violation_description_url,violation_impact,violation_score,web_URL_id,domain_category,screenshot_path,scrape_status,html_file_name,violation_count,wcag_reference,violation_category,supplementary_information
0,1_0,https://www.ki.uni-stuttgart.de/,"[<img src=""https://www.ki.uni-stuttgart.de/img...",image-alt-not-descriptive,Inaccurate or misleading alternative text that...,https://github.com/NadeenAhmad/AccessGuruLLM/b...,critical,5,1,Educational Platforms,/content/screencapture-ki-uni-stuttgart-de-202...,scraped,/content/html_pages_async/www_ki_uni-stuttgart...,2,[1.1.1 Non-text Content],Semantic,"[supplementary_images/image_0.jpeg, supplement..."
1,1_1,https://www.ki.uni-stuttgart.de/,"[<a href=""https://www.ki.uni-stuttgart.de/inst...",link-text-mismatch,Links fail to convey their purpose or are ambi...,https://github.com/NadeenAhmad/AccessGuruLLM/b...,serious,4,1,Educational Platforms,/content/screencapture-ki-uni-stuttgart-de-202...,scraped,/content/html_pages_async/www_ki_uni-stuttgart...,2,"[2.4.4 Link Purpose (In Context), 2.4.9 Link P...",Semantic,The title of the target https://www.ki.uni-stu...


# Save the Result

In [ ]:
violation_df.to_csv("AccessGuruDetectSemantic.csv",index=False)